In [1]:
!pip install beautifulsoup4 as bs4

     |████████████████████████████████| 122kB 6.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [30]:
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import numpy as np

In [31]:
# download data in the url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
rawtext = requests.get(url).text
dsoup = soup(rawtext, "html.parser")

#finding the table element
toronto = dsoup.find('table', class_='wikitable sortable')

In [42]:
#table clean
dt = []
for row in toronto.find_all('tr'):
    col = row.find_all('td')
    if len(col) == 3:
        dt.append((col[0].text.strip(), col[1].text.strip(), col[2].text.strip()))
        
#into dataframe
df = pd.DataFrame(np.asarray(dt))
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

df['Neighborhood'] = df['Neighborhood'].str.replace(']', '')

df.replace('Not assigned', np.nan, inplace=True)

df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,NaN,NaN
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


The next step is to add neighbourhood to a borough where a neighborhood is Not assigned (NaN).

In [43]:
n = len(df.index)
for i in range(0, n):
    if df.iloc[i,1] is not np.nan and df.iloc[i,2] is np.nan:
        df.iloc[i,2] = df.iloc[i,1]
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,NaN,NaN
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Next step: Dropping not assigned boroughs

In [44]:
df.dropna(inplace=True)

Final step in cleaning: grouping neighborhoods in the same borough separated by commas

In [45]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [46]:
print (df.shape)

(103, 3)
